In [2]:
import time
import sagemaker
import os
import json
import boto3

# Retrieve the default bucket
sagemaker_session = sagemaker.Session()
default_bucket = sagemaker_session.default_bucket()

INFO:sagemaker:Created S3 bucket: sagemaker-us-east-1-715445047862


In [3]:
role='arn:aws:iam::715445047862:role/MachineLearningHost'
job_prefix='VGG'
path='neo/%s' % job_prefix

sm = boto3.client('sagemaker')

In [20]:
!curl -O http://download.tensorflow.org/models/official/20181001_resnet/savedmodels/resnet_v1_fp16_savedmodel_NHWC.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 90.7M  100 90.7M    0     0  48.4M      0  0:00:01  0:00:01 --:--:-- 48.4M


In [35]:
!aws s3 cp model.tar.gz s3://$default_bucket/$path/model.tar.gz

upload: ./model.tar.gz to s3://sagemaker-us-east-1-715445047862/neo/VGG/model.tar.gz


TensorFlow: You must specify the name and shape (NHWC format) of the expected data inputs using a dictionary format for your trained model. The dictionary formats required for the console and CLI are different.

Examples for one input:

If using the console, {"input":[1,1024,1024,3]}

If using the CLI, {\"input\":[1,1024,1024,3]}

https://docs.aws.amazon.com/sagemaker/latest/dg/API_InputConfig.html

In [4]:
job_name="%s-%d" % (job_prefix, int(time.time()))
sm.create_compilation_job(
    CompilationJobName=job_name,
    RoleArn=role,
    InputConfig={
        'S3Uri': "s3://%s/%s/model.tar.gz" % (default_bucket, path),
        'DataInputConfig': '{"data":[1,300,300,3]}',
        'Framework': 'TENSORFLOW'
    },
    OutputConfig={
        'S3OutputLocation': "s3://%s/%s/" % (default_bucket, path),
        'TargetDevice': 'jetson_tx2' #'ml_m4'|'ml_m5'|'ml_c4'|'ml_c5'|'ml_p2'|'ml_p3'|'jetson_tx1'|'jetson_tx2'|'rasp3b'|'deeplens'
    },
    StoppingCondition={
        'MaxRuntimeInSeconds': 300
    }
)

{'CompilationJobArn': 'arn:aws:sagemaker:us-east-1:715445047862:compilation-job/VGG-1552396858',
 'ResponseMetadata': {'RequestId': '3e05b4a0-97e4-46e9-aceb-7a003249466b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3e05b4a0-97e4-46e9-aceb-7a003249466b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '95',
   'date': 'Tue, 12 Mar 2019 13:20:58 GMT'},
  'RetryAttempts': 0}}

In [5]:
resp = sm.describe_compilation_job(
    CompilationJobName=job_name
)
if resp['CompilationJobStatus'] == 'FAILED':
    print(resp['FailureReason'])
elif resp['CompilationJobStatus'] == 'INPROGRESS':
    print('COMPILING')
else:
    print('OK')


OK
